# Query Six B Preprocessing

####  Are there any other listings by the same host that can be suggested? Display listing’s name, url, description, host’s name, reviewer name, whether previously booked, availability days, minimum and maximum nights booking allowed. 

#### Goal: Clean the data files in prepartion for loading into Cassandra on Azure

In [37]:
#Imports
import numpy as np
import pandas as pd

In [50]:
#Uploading reviews file
reviews = pd.read_csv("q6_comb_reviews.csv")
reviews.head()

,Unnamed: 0,listing_id,id,reviewer_id,reviewer_name,review_month
0,0,12899,24767,69327,Stuart,1
1,1,12899,29230,72846,John,3
2,2,12899,29806,84196,Lois,3
3,3,12899,32572,89114,Troy,3
4,4,12899,32862,100318,Cathy,4


In [ ]:
'''Steps:
    1. Load in the combined reviews table and filter it to only have listing_id, reviewer_id, and count(reviews) > 3
    2. Change the name of listing_id to "thrice_listing_id"
    3. Add host_id column to reviews table by joining on listings file
    4. Left join the listings table to reviews again, joining by host_id and pulling in the listing info
    5. Join the availability days file to the reviews file above
    6. Create logic to test if the reviewer has previously stayed at the booking and add column'''

In [41]:
#Loading in the combined reviews table, grouping to get review counts, and filtering by count > 3
grouped_listings = reviews.groupby(['listing_id', 'reviewer_id', 'reviewer_name'])['id'].count().to_frame().reset_index()
grouped_listings = grouped_listings.rename({"id":"review_counts"}, axis = 1)
thrice_reviewed = grouped_listings.loc[grouped_listings['review_counts'] >= 3]
thrice_reviewed.head()

,listing_id,reviewer_id,reviewer_name,review_counts
231,3021,82971860,Jason,3
1635,9140,54090633,Shijuana,3
1969,12899,791723,Barbara,3
1987,12899,1227853,Beth,5
2065,12899,3132153,Dinah,3


In [42]:
#Updating the listing_id column name
thrice_reviewed = thrice_reviewed.rename({'listing_id':'thrice_rev_listing_id'}, axis = 1)
thrice_reviewed

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts
231,3021,82971860,Jason,3
1635,9140,54090633,Shijuana,3
1969,12899,791723,Barbara,3
1987,12899,1227853,Beth,5
2065,12899,3132153,Dinah,3
...,...,...,...,...
2511389,789101243357990362,485134984,Hernan,6
2511777,792942008294249400,485134984,Hernan,4
2512605,798843846505276637,454135038,Danielle,3
2512897,802203851341181214,20189866,Moe,3


In [43]:
#Uploading all listings files and merging into a single file
la_listing = pd.read_csv('la_listings.csv')
pdx_listing = pd.read_csv('PDX_listings.csv')
sd_listing = pd.read_csv('sd_listings.csv')
salem_listing = pd.read_csv('salem_listings.csv')

df_list = [la_listing, pdx_listing, sd_listing, salem_listing]
all_listings = pd.concat(df_list)

In [44]:
#Joining the host_id column on the thrice reviewed table from all_listings
thrice_reviewed_host = pd.merge(thrice_reviewed, all_listings[['id','host_id']], how = 'left', left_on = 'thrice_rev_listing_id', right_on = 'id')
thrice_reviewed_host = thrice_reviewed_host.loc[:,thrice_reviewed_host.columns!='id']
thrice_reviewed_host.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id
0,3021,82971860,Jason,3,3415
1,9140,54090633,Shijuana,3,28350
2,12899,791723,Barbara,3,49682
3,12899,1227853,Beth,5,49682
4,12899,3132153,Dinah,3,49682


In [45]:
#Joining the listing information onto the thrice_reviewed_host table
thrice_listing = pd.merge(thrice_reviewed_host, all_listings[['id','name','listing_url','description','host_name','host_id','minimum_nights','maximum_nights']], how = 'left', on = 'host_id')
thrice_listing.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id,id,name,listing_url,description,host_name,minimum_nights,maximum_nights
0,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730
1,3021,82971860,Jason,3,3415,38365,Hollywood Hills Private Br Suite,https://www.airbnb.com/rooms/38365,Laurel Canyon is a very musically historic seg...,Nataraj,30,730
2,3021,82971860,Jason,3,3415,8224450,Zen Hollywood Hills Unit,https://www.airbnb.com/rooms/8224450,Beautiful private apt wing guest house of larg...,Nataraj,30,730
3,9140,54090633,Shijuana,3,28350,9140,City Place Lofts,https://www.airbnb.com/rooms/9140,Shared space - private room with its own entra...,Wendell,31,1125
4,12899,791723,Barbara,3,49682,12899,"Alberta Arts 2 bedroom suite, charming 1906 house",https://www.airbnb.com/rooms/12899,"Please know that we are fully vaccinated, will...",Ali And David,3,730


##  LOAD KEVIN TABLE OF AVAILABILITY INTO THE BELOW BLOCK INSTEAD OF DUMMY TABLE
#### Cleaning may be necessary to get Kevin's table into my preferred format

In [46]:
#Loading in table with 2 columns - listing_id and avail_month
#listing_id is the primary key, avail month is a column containing a string with all the months of availability for the property 
#e.g., '1,2,3' for Jan, Feb, Mar. Kevin wrote the code to execute the availability calculation. 
avail_table = pd.read_csv('dummy_availability_table.csv')
avail_table.head()

,Unnamed: 0,listing_id,avail_months
0,0,12899,"1,2,3"
1,1,789798,"1,2,3"
2,2,801318,"1,2,3"
3,3,808393,"1,2,3"
4,4,838961,"1,2,3"


In [48]:
#Match the availability table to the thrice listing table, removing listing_id from join & renaming columns for more obvious descriptions
thrice_listing_avail = pd.merge(thrice_listing, avail_table[['avail_months', 'listing_id']], how = "left", left_on = "id", right_on = 'listing_id')
thrice_listing_avail = thrice_listing_avail.loc[:, thrice_listing_avail.columns!='listing_id']
thrice_listing_avail = thrice_listing_avail.rename({'id':'listing_id', 'name':'listing_name', 'description':'listing_description'}, axis = 1)
thrice_listing_avail.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id,listing_id,listing_name,listing_url,listing_description,host_name,minimum_nights,maximum_nights,avail_months
0,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6"
1,3021,82971860,Jason,3,3415,38365,Hollywood Hills Private Br Suite,https://www.airbnb.com/rooms/38365,Laurel Canyon is a very musically historic seg...,Nataraj,30,730,"1,2,3"
2,3021,82971860,Jason,3,3415,8224450,Zen Hollywood Hills Unit,https://www.airbnb.com/rooms/8224450,Beautiful private apt wing guest house of larg...,Nataraj,30,730,"1,2,3"
3,9140,54090633,Shijuana,3,28350,9140,City Place Lofts,https://www.airbnb.com/rooms/9140,Shared space - private room with its own entra...,Wendell,31,1125,"7,8,9"
4,12899,791723,Barbara,3,49682,12899,"Alberta Arts 2 bedroom suite, charming 1906 house",https://www.airbnb.com/rooms/12899,"Please know that we are fully vaccinated, will...",Ali And David,3,730,"1,2,3"


In [68]:
'''Create logic to test if the reviewer has previously stayed at the booking
and add column'''

'Create logic to test if the reviewer has previously stayed at the booking\nand add column'

In [92]:
# Merging listing_id and reviewer_id columns in thrice_listing_avail
thrice_listing_avail['listing_reviewer'] = thrice_listing_avail['listing_id'].astype(str) + thrice_listing_avail['reviewer_id'].astype(str)
thrice_listing_avail.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id,listing_id,listing_name,listing_url,listing_description,host_name,minimum_nights,maximum_nights,avail_months,listing_reviewer
0,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860
1,3021,82971860,Jason,3,3415,38365,Hollywood Hills Private Br Suite,https://www.airbnb.com/rooms/38365,Laurel Canyon is a very musically historic seg...,Nataraj,30,730,"1,2,3",3836582971860
2,3021,82971860,Jason,3,3415,8224450,Zen Hollywood Hills Unit,https://www.airbnb.com/rooms/8224450,Beautiful private apt wing guest house of larg...,Nataraj,30,730,"1,2,3",822445082971860
3,9140,54090633,Shijuana,3,28350,9140,City Place Lofts,https://www.airbnb.com/rooms/9140,Shared space - private room with its own entra...,Wendell,31,1125,"7,8,9",914054090633
4,12899,791723,Barbara,3,49682,12899,"Alberta Arts 2 bedroom suite, charming 1906 house",https://www.airbnb.com/rooms/12899,"Please know that we are fully vaccinated, will...",Ali And David,3,730,"1,2,3",12899791723


In [69]:
# Merging listing_id and reviewer_id columns in thrice_listing_avail
reviews['listing_reviewer'] = reviews['listing_id'].astype(str) + reviews['reviewer_id'].astype(str)
reviews.head()

,Unnamed: 0,listing_id,id,reviewer_id,reviewer_name,review_month,listing_reviewer
0,0,12899,24767,69327,Stuart,1,1289969327
1,1,12899,29230,72846,John,3,1289972846
2,2,12899,29806,84196,Lois,3,1289984196
3,3,12899,32572,89114,Troy,3,1289989114
4,4,12899,32862,100318,Cathy,4,12899100318


In [103]:
#Testing if concat listing & reviewer id in thrice_listing_avail is also in the reviews tables via a merge
booking_reminder = pd.merge(thrice_listing_avail, reviews[['listing_reviewer', 'listing_id']], how = "left", on = 'listing_reviewer')
booking_reminder.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id,listing_id_x,listing_name,listing_url,listing_description,host_name,minimum_nights,maximum_nights,avail_months,listing_reviewer,listing_id_y
0,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860,3021.0
1,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860,3021.0
2,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860,3021.0
3,3021,82971860,Jason,3,3415,38365,Hollywood Hills Private Br Suite,https://www.airbnb.com/rooms/38365,Laurel Canyon is a very musically historic seg...,Nataraj,30,730,"1,2,3",3836582971860,NaN
4,3021,82971860,Jason,3,3415,8224450,Zen Hollywood Hills Unit,https://www.airbnb.com/rooms/8224450,Beautiful private apt wing guest house of larg...,Nataraj,30,730,"1,2,3",822445082971860,NaN


In [104]:
#Changing listing_id_y (the listing id from the reviews table) column name and setting to 1 or 0
booking_reminder = booking_reminder.rename({'listing_id_y':'prev_booked_by_reviewer', 'listing_id_x':'reccommended_listing_id'}, axis = 1)
booking_reminder['prev_booked_by_reviewer'] = np.where(booking_reminder['prev_booked_by_reviewer'].isnull(),0,1) 
booking_reminder.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id,reccommended_listing_id,listing_name,listing_url,listing_description,host_name,minimum_nights,maximum_nights,avail_months,listing_reviewer,prev_booked_by_reviewer
0,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860,1
1,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860,1
2,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",302182971860,1
3,3021,82971860,Jason,3,3415,38365,Hollywood Hills Private Br Suite,https://www.airbnb.com/rooms/38365,Laurel Canyon is a very musically historic seg...,Nataraj,30,730,"1,2,3",3836582971860,0
4,3021,82971860,Jason,3,3415,8224450,Zen Hollywood Hills Unit,https://www.airbnb.com/rooms/8224450,Beautiful private apt wing guest house of larg...,Nataraj,30,730,"1,2,3",822445082971860,0


In [110]:
#Remove duplicate rows in the booking_reminder table
booking_reminder_final = booking_reminder.drop_duplicates()
booking_reminder_final = booking_reminder_final.loc[:, booking_reminder_final.columns!="listing_reviewer"]
booking_reminder_final.head()

,thrice_rev_listing_id,reviewer_id,reviewer_name,review_counts,host_id,reccommended_listing_id,listing_name,listing_url,listing_description,host_name,minimum_nights,maximum_nights,avail_months,prev_booked_by_reviewer
0,3021,82971860,Jason,3,3415,3021,Large Furnished Bedroom in Hollywood Hills House,https://www.airbnb.com/rooms/3021,Great style and comfort in the spacious 4000 s...,Nataraj,30,730,"4,5,6",1
3,3021,82971860,Jason,3,3415,38365,Hollywood Hills Private Br Suite,https://www.airbnb.com/rooms/38365,Laurel Canyon is a very musically historic seg...,Nataraj,30,730,"1,2,3",0
4,3021,82971860,Jason,3,3415,8224450,Zen Hollywood Hills Unit,https://www.airbnb.com/rooms/8224450,Beautiful private apt wing guest house of larg...,Nataraj,30,730,"1,2,3",0
5,9140,54090633,Shijuana,3,28350,9140,City Place Lofts,https://www.airbnb.com/rooms/9140,Shared space - private room with its own entra...,Wendell,31,1125,"7,8,9",1
8,12899,791723,Barbara,3,49682,12899,"Alberta Arts 2 bedroom suite, charming 1906 house",https://www.airbnb.com/rooms/12899,"Please know that we are fully vaccinated, will...",Ali And David,3,730,"1,2,3",1


In [111]:
#Saving output file
booking_reminder_final.to_csv("q6_b_reccommendations.csv")